In [1]:
from init import *

autotime not loaded.
autotime not loaded.
black loaded.
black loaded.


<IPython.core.display.Javascript object>

In [2]:
# Import modules for training

import rasa_nlu
import rasa_core
import spacy

# RASA NLU
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config


# RASA Core
from rasa_core.policies import FallbackPolicy, MemoizationPolicy, KerasPolicy
from rasa_core.agent import Agent
import json

<IPython.core.display.Javascript object>

In [3]:
# Writing various intents with examples to nlu.md file

nlu_md = """
## intent:greet
- hey
- hello there
- hi
- hello there
- good morning
- good evening
- moin
- hey there
- let's go
- hey dude
- goodmorning
- goodevening
- good afternoon

## intent:goodbye
- good by
- cee you later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- see you later

## intent:order
- Are you open
- Can I order
- I want to order
- Will you deliver ?
- I want to place order
- Is food available now ?
- Can I order now ?

## intent:choosing_item
- 1
- 2
- 3
- 4
- 5
- 6
- 7

## intent:order_enquiry
- How long will it take ?
- How much time ?
- When can I expect the order
- Will it take long ?
- I am waiting for the delivery
- Who will deliver 
- I am hungry

## intent:complain
- food was bad
- poor quality
- It was bad
- very bad taste
- poor taste
- there was a fly in it
- It was not hygienic
- Not clean
- poor packaging
- I won't order again
- bad
- terrible
- wrong food delivered
- I didn't order this


## intent:suggest
- Make it more spicy
- You can improve actually
- You can improve the taste
- you can expand the menu I guess
- Add more dishes to the menu
- speed delivery would be better
- try to reduce time
- Add continental dishes too!
- Hope you'll take less time hereafter
"""

%store nlu_md >nlu.md

Writing 'nlu_md' (str) to file 'nlu.md'.


<IPython.core.display.Javascript object>

In [4]:
# Writing stories and saving it in the stories.md file

stories_md = """
##  Greeting path             
* greet              
  - utter_greet

## Order path
* order
  - utter_menu
* choosing_item
  - utter_order_received

## Enquires path
* order_enquiry
  - utter_enquiry

## complain path
* complain
  - utter_receive_complaint


## suggestion path
* suggest
  - utter_take_suggestion


## say goodbye
* goodbye
  - utter_goodbye
"""

%store stories_md >stories.md

Writing 'stories_md' (str) to file 'stories.md'.


<IPython.core.display.Javascript object>

In [5]:
# Writing all the intents,slots,entities,actions and templates to domain.yml

domain_yml = """
intents:
- greet
- goodbye
- order
- choosing_item
- order_enquiry
- complain
- suggest

slots:
  group:
    type: text

entities:
- group

actions:
- utter_greet
- utter_menu
- utter_order_received
- utter_enquiry
- utter_receive_complaint
- utter_take_suggestion
- utter_goodbye


templates:
  utter_greet:
  - text: "Hey! Welcome to Biker's cafe . How may I help you ?"

  utter_menu:
  - text: "Yes, we are open 24 X 7. Here's our menu. 1. Chicken popcorn  2. White sauce pasta 3. Macroni  4. Lasagna  5. Pizza  6. Taco  7. Garlic breadsticks  8. coke (500 ml). Enter  1 if you want popcorn, 5 for Pizza and so on"

  utter_order_received:
  - text: "Great choice ! Your order has been confirmed . Wait while our top chefs prepare your delicious food !"

  utter_goodbye:
  - text: " Thankyou for choosing us. Waiting for your next visit "

  utter_enquiry:
  - text: " Your food is being prepared. It usually takes 25-30 minutes "

  utter_receive_complaint:
  - text: " Sorry that you faced trouble. We have noted your issues and it shall not be repeated again"

  utter_take_suggestion:
  - text: " Sure , we shall take into account . Thanks for suggestion "

"""

%store domain_yml >domain.yml

Writing 'domain_yml' (str) to file 'domain.yml'.


<IPython.core.display.Javascript object>

In [6]:
# Adding the NLU components to the pipeline in config.yml file

mconfig = """
language: "en_core_web_md"

pipeline:
- name: "nlp_spacy"                   # loads the spacy language model
- name: "tokenizer_spacy"             # splits the sentence into tokens
- name: "ner_crf"                     # uses the pretrained spacy NER model
- name: "intent_featurizer_spacy"     # transform the sentence into a vector representation
- name: "intent_classifier_sklearn"   # uses the vector representation to classify using SVM
- name: "ner_synonyms"                # trains the synonyms
"""

%store mconfig >config.yml

Writing 'mconfig' (str) to file 'config.yml'.


<IPython.core.display.Javascript object>

In [7]:
class ChatBot:
    def __init__(
        self,
        ftrain="nlu.md",
        fconfig="config.yml",
        fdomain="domain.yml",
        fstories="stories.md",
        dir_model="./models/nlu",
        dir_dialogue="models/dialogue",
    ):
        self.domain = fdomain
        self.stories = fstories
        self.configuration = config.load(fconfig)
        self.dialogue_directory = dir_dialogue
        self.model_directory = dir_model
        self.nlu_data = load_data(ftrain)

    def train(
        self, policies=[MemoizationPolicy(), KerasPolicy()], epochs=200, vsplit=0.0
    ):
        self.trainer = Trainer(self.configuration)
        self.interpreter = self.trainer.train(self.nlu_data)
        self.nlu_directory = self.trainer.persist(
            self.model_directory, fixed_model_name="current"
        )

        agent = Agent(self.domain, policies=policies)
        self.core_data = agent.load_data(self.stories)
        agent.train(self.core_data, validation_split=vsplit, epochs=epochs)
        agent.persist(self.dialogue_directory)

    def run(self):
        agent = Agent.load(self.dialogue_directory, interpreter=self.nlu_directory)

        print("Your bot is ready to talk! Type your messages here or send 'stop'")

        while True:
            a = input()
            if a == "stop":
                break

            responses = agent.handle_message(a)
            for response in responses:
                print(response["text"])

    def test_intent(self, msg="good night"):
        message = self.interpreter.parse(msg)
        print(json.dumps(message, indent=2))

Using TensorFlow backend.


<IPython.core.display.Javascript object>

In [8]:
bot = ChatBot()
bot.train()

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.1s finished
Processed Story Blocks:   0%|          | 0/6 [00:00<?, ?it/s, # trackers=18]

Fitting 2 folds for each of 6 candidates, totalling 12 fits


Processed trackers: 100%|██████████| 248/248 [00:01<00:00, 205.55it/s, # actions=351]
Processed actions: 351it [00:00, 1679.85it/s, # examples=351]
Processed trackers: 100%|██████████| 248/248 [00:01<00:00, 221.03it/s, # actions=351]


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, 5, 18)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                6528      
_________________________________________________________________
dense_1 (Dense)              (None, 9)                 297       
_________________________________________________________________
activation_1 (Activation)    (None, 9)                 0         
Total params: 6,825.0
Trainable params: 6,825.0
Non-trainable params: 0.0
_________________________________________________________________
Epoch 1/200
351/351 [==========================

<IPython.core.display.Javascript object>

In [9]:
bot.test_intent()

{
  "intent": {
    "name": "goodbye",
    "confidence": 0.5544559217164021
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "goodbye",
      "confidence": 0.5544559217164021
    },
    {
      "name": "greet",
      "confidence": 0.33705069548867556
    },
    {
      "name": "complain",
      "confidence": 0.040135029263428476
    },
    {
      "name": "order_enquiry",
      "confidence": 0.02050827605825517
    },
    {
      "name": "suggest",
      "confidence": 0.019575600271982567
    },
    {
      "name": "choosing_item",
      "confidence": 0.014923551577718087
    },
    {
      "name": "order",
      "confidence": 0.013350925623538115
    }
  ],
  "text": "good night"
}


<IPython.core.display.Javascript object>

In [10]:
bot.run()

Your bot is ready to talk! Type your messages here or send 'stop'


 hi


Hey! Welcome to Biker's cafe . How may I help you ?


 need food


Yes, we are open 24 X 7. Here's our menu. 1. Chicken popcorn  2. White sauce pasta 3. Macroni  4. Lasagna  5. Pizza  6. Taco  7. Garlic breadsticks  8. coke (500 ml). Enter  1 if you want popcorn, 5 for Pizza and so on


 7


Great choice ! Your order has been confirmed . Wait while our top chefs prepare your delicious food !


 stop


<IPython.core.display.Javascript object>